In [1]:
! wget "https://jimut123.github.io/courses/vision/assets/sonar.csv"

--2024-01-30 02:15:50--  https://jimut123.github.io/courses/vision/assets/sonar.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving jimut123.github.io (jimut123.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to jimut123.github.io (jimut123.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87776 (86K) [text/csv]
Saving to: ‘sonar.csv’

sonar.csv           100%[===================>]  85.72K  --.-KB/s    in 0.01s   

2024-01-30 02:15:51 (8.35 MB/s) - ‘sonar.csv’ saved [87776/87776]



In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('sonar.csv')

In [4]:
df.shape

(207, 61)

In [5]:
X = df.to_numpy()[:,:60]
Y = df.to_numpy()[:,60:]

In [6]:
print(X.shape)
print(Y.shape)

(207, 60)
(207, 1)


In [7]:
 def get__number_from_pal(str_num):
    return np.fromiter((int(bit) for bit in str_num), dtype=np.int8)

In [8]:
# Generating palindrome data

def is_pal(X):
    return X == X[::-1]

def generate_10_bit_binary_numbers():
    binary_numbers = []

    for i in range(2**10):
        binary_str = format(i, '010b')
        binary_numbers.append(binary_str)

    return binary_numbers


# Generate and store all 10-bit binary numbers
binary_numbers_list = generate_10_bit_binary_numbers()


Y_VALUES = []
X_VALUES = []

# Print and/or use the generated binary numbers as needed
for binary_number in binary_numbers_list:
    #print(binary_number)
    if is_pal(binary_number) == True:
        X_VALUES.append(get__number_from_pal(binary_number))
        Y_VALUES.append(1)
    else:
        X_VALUES.append(get__number_from_pal(binary_number))
        Y_VALUES.append(0)

In [9]:
X = np.array(X_VALUES)
Y = np.array(Y_VALUES)
print("X = ",X.shape)
print("Y = ",Y.shape)

X =  (1024, 10)
Y =  (1024,)


In [10]:
Y = Y.reshape(1024,1)

In [11]:
Y.shape

(1024, 1)

In [12]:
# prompt: generate a blank class definition for a MLP in numpy

class MLP:
  def __init__(self, input, hidden):
    self.w1 = np.random.rand(input, hidden)
    # self.w1 = np.random.rand(10, hidden)
    self.w2 = np.random.rand(hidden, 1)

    self.hidden = hidden
    self.input = input

  def relu(self, x):
    return np.maximum(x,0)

  def relu_prime(self,x):
    if x>0:
      return 1
    else:
      return 0

  def sigmoid(self,x):
    # return 1 / (1 + np.exp(-x))
    return np.exp(-np.logaddexp(0, -x))


  def sigmoid_prime(self,x):
    z = self.sigmoid(x)
    return z*(1-z)

  def forward(self, x):
    # forward
    z1 = self.w1.T @ x
    a1 = self.relu(z1)

    z2 = self.w2.T @ a1
    a2 = self.sigmoid(z2)

    return a2

  def forward_backward(self, x, y, lr):

    # forward
    z1 = self.w1.T @ x
    a1 = self.relu(z1)

    z2 = self.w2.T @ a1
    a2 = self.sigmoid(z2)

    # backprop
    self.dw1 = np.zeros_like(self.w1)
    self.dw2 = np.zeros_like(self.w2)

    L = -y*np.log(a2 + 1e-6) - (1-y)*np.log(1-a2 + 1e-6)
    # da2 = -y/a2 + (1-y)/(1-a2)

    for i in range(self.hidden):
      self.dw2[i,0] = (-y[0,0]*(1-a2[0,0]) + (1- y[0,0])*a2[0,0]) * a1[i,0]
      #self.dw2[i,0] = da2[0,0] * self.sigmoid_prime(z2[0,0]) * a1[i,0]

    for i in range(self.input):
      for j in range(self.hidden):
        # self.dw1[i,0] = da2[0,0] * self.sigmoid_prime(z2[0,0]) * self.w2[j,0] * self.relu_prime(z1[j,0]) * x[i,0]
        self.dw1[i,j] = (-y[0,0]*(1-a2[0,0]) + (1- y[0,0])*a2[0,0])  * self.w2[j,0] * self.relu_prime(z1[j,0]) * x[i,0]

    # update
    self.w1 = self.w1 - lr*self.dw1
    self.w2 = self.w2 - lr*self.dw2

    # print (np.linalg.norm(self.dw1))

    return a2 , L


In [13]:
net = MLP(10, 16)

In [14]:
# eval
def inference():
  print ('inference')
  correct = 0

  for i in range(len(X)):
    x = X[i:i+1,:].T
    y = Y[i,0]

    y_pred = net.forward(x)

    y_pred = y_pred[0,0]

    # print (y_pred)

    if y_pred < 0.5:
      y_pred = 0
    else:
      y_pred = 1

    if (y_pred == y):
      correct +=1

  print (correct)

In [17]:
for epoch in range(1000):

  total = 0

  for i in range(len(X)):
    x = X[i:i+1,:].T
    # y = Y[i:i+1,:]
    y = Y[i:i+1,:]
    
#     print("Shape of X = ",x.shape," shape of y = ",y.shape)

    y_pred, L = net.forward_backward(x,y,1e-2)

    # print(y_pred, '- gt:', y)
    # print(y)
    # print (L)
    # print("=====")

    total += L

  print (epoch , ' loss: ', total)
  print ("====")

  if epoch%20==0:
    inference()


0  loss:  [[7.88910273]]
====
inference
1024
1  loss:  [[7.9273436]]
====
2  loss:  [[7.76375454]]
====
3  loss:  [[7.62785961]]
====
4  loss:  [[7.48014795]]
====
5  loss:  [[7.36396099]]
====
6  loss:  [[7.21812797]]
====
7  loss:  [[7.11273735]]
====
8  loss:  [[6.98889939]]
====
9  loss:  [[6.87625184]]
====
10  loss:  [[6.76372958]]
====
11  loss:  [[6.6587437]]
====
12  loss:  [[6.55043762]]
====
13  loss:  [[6.45460097]]
====
14  loss:  [[6.35163658]]
====
15  loss:  [[6.2590139]]
====
16  loss:  [[6.17569379]]
====
17  loss:  [[6.0890502]]
====
18  loss:  [[5.99734602]]
====


KeyboardInterrupt: 

In [ ]:
y_pred_list = []

for i in range(len(X)):
    x = X[i:i+1,:].T
    y = Y[i,0]

    y_pred = net.forward(x)

    y_pred = y_pred[0,0]

    y_pred_list.append(y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y, y_pred_list)

0.5

In [ ]:
# 100  loss:  [[79.19130256]]  - using randn as weights
# 100  loss:  [[26.3380848]] - using rand as weights

In [ ]:
np.sum(Y==1)

111

In [ ]:
np.sum(Y==0)

96

In [ ]:
len(X)
# 96 correct beginning

207

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Create the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(16,), max_iter=1000, random_state=42, solver='sgd', learning_rate_init = 1e-2)

# Train the model
mlp_model.fit(X, Y)

# Make predictions on the test set
y_pred = mlp_model.predict(X)

# Calculate accuracy
accuracy = accuracy_score(Y, y_pred)
print(f"Accuracy: {accuracy}")

# You can also access the trained model parameters
print("Trained model parameters:")
print("Input layer size:", mlp_model.n_features_in_)
print("Output layer size:", mlp_model.n_outputs_)
# print("Hidden layer sizes:", mlp_model.hidden_layer_sizes_)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 1.0
Trained model parameters:
Input layer size: 10
Output layer size: 1


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
992/1024

0.96875